<a href="https://colab.research.google.com/github/rai-sandeep/ai-colab-notebooks/blob/main/Training-Stable-V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [21]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
trained_model_id = "falcon-7b-instruct-sharded-trained-v1"
ds_id = "rai-sandeep/dataset_format_v1"
example_ques = "Generate a White Paper ABSTRACT on Modernizing Enterprise Systems."
#example_ques2 = "Generate a White Paper MAINCONTENT on Bitcoin Mining."
max_tok = 200

save_trained_model=False
%env HF_TOKEN=<ENTER-HUGGINGFACE-TOKEN-HERE>

env: HF_TOKEN=<ENTER-HUGGINGFACE-TOKEN-HERE>


In [22]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Import

In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer

# Load Quantized Model

In [24]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [25]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [26]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [27]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


# Prepare Data

In [28]:
from datasets import load_dataset

data = load_dataset(ds_id)

ds_len=len(data['train'])
rows_sel=data['train'].select(range(1))
rows_sel['template']

  0%|          | 0/1 [00:00<?, ?it/s]

['This white paper examines [Topic/Issue] and provides insights into [Key Points/Findings]. It explores [Main Theme/Techniques] and highlights [Benefits/Challenges]. The analysis presented here acts as a guideline for [Target Audience] to make strategic and informed decisions on [Action/Implementation]. By delving into [Key Concepts/Approaches], this paper aims to assist [Target Audience] in [Objective/Goal]. The findings discussed herein contribute to the ongoing conversation surrounding [Topic/Issue] and offer practical recommendations for [Action/Implementation].']

In [29]:
tokenizer.pad_token = tokenizer.eos_token

# prompt_template = "### Instruction: {prompt}\n### Response:"

train_dataset = data['train'].select(range(ds_len)).map(lambda x: {"input_text": "Format for " + x['doctype'] + " " + x['section'] + ":\n" + x['template']})

# Tokenize the datasets
train_encodings = tokenizer(train_dataset['input_text'], max_length=256, truncation=True, padding=True, return_tensors='pt')

In [30]:
  class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [31]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)

# Example Before Fine Tuning

In [32]:
def generate(when_id):

  doc_type = "White Paper"
  topic = "Infosys Watch Tower"
  keywords = "Fire and smoke detection, Birds monitoring, Unattended cabin baggage detection, Vegetation monitoring"
  content_to_use = "Infosys Watch Tower enables real-time monitoring of situations, which is an important aspect of operations, specifically in the aviation industry. The need for monitoring increases exponentially at airports where crowd management and security are amplified. Infosys Watch Tower mitigates security threats and tracks cargo, thereby ensuring smooth operations. The platform onboards different sensors such as camera, drone, and light detection and ranging (LiDAR) to detect and gather information about incidents, while streamlining business flows at airports and on runways. Infosys Watch Tower uses drones to inspect aircraft and sonar for Non-destructive Testing (NDT) for identifying and detecting material damage. It is scalable to support thousands of sensors along with real-time processed feeds based on sensor configuration and monitoring plans. Infosys Watch Tower leverages cutting-edge 5G technology for aviation enterprises to create low-code / no-code AI workflows for installed sensors. These workflows are easily configurable and can be deployed across various edges. The platform can be extended to multiple AI domains such as computer vision, and across multiple sub-domains such as retail, facility management, and security, which are ancillary businesses in aviation. The platform offers a unified view of inferences, reports, monitoring plans and alerts via reports and analytics. It provides management with 360-degree visibility into different incidents and events. Apart from serving as a smart assistant for security personnel and a guide to the operations manager, the data collected as part of 24x7 monitoring enhances overall security and maintenance."

  doc_content=""
  doc_parts=["Abstract","Introduction","Problem Statement","Methodology","Findings and Analysis","Recommendations","Conclusion"]

  for doc_part in doc_parts:
    prompt = f"Generate a {doc_type} {doc_part} on {topic}."
    if keywords.strip() != "":
      prompt += f"\nInclude the following keywords: {keywords}."
    if content_to_use.strip() != "":
      prompt += f"\nHere is some context about {topic}: {content_to_use}.\n"

    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=max_tok, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

    temp_content=tokenizer.decode(output[0], skip_special_tokens=True)
    temp_content=temp_content.replace(prompt, "")
    doc_content+=f"\n## {doc_part}  \n{temp_content}  \n"

  print(doc_content)

  f = open(topic.replace(" ", "-")+"-"+when_id+".md", "w")
  f.write(doc_content)
  f.close()

In [33]:
generate("before")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1264: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



## Abstract  

Modernizing enterprise systems is a critical challenge for businesses today. This white paper explores the current state of enterprise systems and the opportunities for improvement. It examines the drivers for change, the challenges and barriers to modernization, and the benefits that can be achieved. The paper also provides practical guidance on how to approach modernization, including strategies for successful implementation and ongoing maintenance.  

## Introduction  

In today's fast-paced digital world, businesses need to keep up with the latest trends and technologies to remain competitive. This white paper explores the challenges and opportunities that modern enterprise systems face, and how businesses can leverage emerging technologies to streamline their operations and drive innovation.  

## Problem Statement  

Modernizing enterprise systems is a complex challenge that organizations face as they strive to keep pace with rapidly evolving technology landscapes

# Training

In [34]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=50,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        # max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

#import gc
#del train_encodings, data
#gc.collect()
#torch.cuda.empty_cache()

trainer.train()

<ipython-input-30-bd500a371ea7>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,0.474100
2,0.474100
3,0.472500
4,0.467900
5,0.458500
6,0.445400
7,0.429000
8,0.410000
9,0.388900
10,0.368200


TrainOutput(global_step=50, training_loss=0.16466441610828042, metrics={'train_runtime': 274.5782, 'train_samples_per_second': 1.275, 'train_steps_per_second': 0.182, 'total_flos': 1741947043200000.0, 'train_loss': 0.16466441610828042, 'epoch': 50.0})

# Example After Fine Tuning

In [35]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): RWForCausalLM(
      (transformer): RWModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x DecoderLayer(
            (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
            (self_attention): Attention(
              (maybe_rotary): RotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Parameter

In [36]:
generate("after")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



## Abstract  

In this white paper, we explore the challenges and opportunities associated with modernizing enterprise systems. We highlight the benefits such as increased efficiency, improved decision-making, and enhanced competitiveness. We propose practical strategies and approaches for implementing modernized systems, including the adoption of cutting-edge technologies, the implementation of innovative processes, and the implementation of data-driven analytics. By focusing on the advantages and leveraging the latest technologies, organizations can successfully modernize their systems and achieve their business objectives.  

## Introduction  

In today's fast-paced digital landscape, enterprises across industries are facing challenges in effectively and efficiently managing their operations. This white paper explores the modernizing trends of enterprise systems and the potential benefits in streamlining processes, improving decision-making, and enhancing overall performance. By em

# Save the Model

In [37]:
if (save_trained_model):
  trainer.save_model(trained_model_id)
  !huggingface-cli login --token $HF_TOKEN
  model.push_to_hub("rai-sandeep/"+trained_model_id, create_pr=1)

In [38]:
def prompt_and_generate():

  doc_type = input("What type of document do you want to generate? (Enter White Paper or Press Release)\n")
  topic = input(f"On what topic do you want to generate the {doc_type}?\n")
  keywords = input(f"Enter any keywords that should be part of the {doc_type} in comma separated format. If none, press Enter.\n")
  content_to_use = input(f"Enter any content that should be used for the {doc_type}. If none, press Enter.\n")

  doc_content=""
  doc_parts=["Abstract","Introduction","Problem Statement","Methodology","Findings and Analysis","Recommendations","Conclusion"]

  for doc_part in doc_parts:
    prompt = f"Generate a {doc_type} {doc_part} on {topic}."
    if keywords.strip() != "":
      prompt += f"\nInclude the following keywords: {keywords}."
    if content_to_use.strip() != "":
      prompt += f"\nHere is some context about {topic}: {content_to_use}.\n"

    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=max_tok, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

    temp_content=tokenizer.decode(output[0], skip_special_tokens=True)
    temp_content=temp_content.replace(prompt, "")
    doc_content+=f"\n## {doc_part}  \n{temp_content}  \n"

  print(doc_content)

  f = open(topic.replace(" ", "-")+".md", "w")
  f.write(doc_content)
  f.close()

In [ ]:
#prompt_and_generate()

What type of document do you want to generate? (Enter White Paper or Press Release)
White Paper
On what topic do you want to generate the White Paper?
Smarter way to build system resilience
Enter any keywords that should be part of the White Paper in comma separated format. If none, press Enter.
Infrastructure, chaos, regression
Enter any content that should be used for the White Paper. If none, press Enter.
How Chaos Engineering is Different from Traditional Testing Practices • Performance testing – It baselines application performance under a defined load in favorable environmental conditions. The main objective is to check how the system performs when the application is up and running without any severe functional defects in an environment comparable to the production environment. The potential disruptors uncovered during the performance tests are due to certain load conditions on the application. • Disaster recovery testing – This process ensures that an organization can restore it

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
